## l_______________________________________________________________________

In [ ]:
import json
import os
import pandas as pd
from genData import load_model
import re
import numpy as np
import pickle

from importlib import reload
import mainlib
reload(mainlib)
import A_base as ab
reload(ab)
from  mainlib import fasorial, lista, sdict, mm, mm_sep


import collections
import of_proc
reload(of_proc)
from of_proc import *

# Informações

In [ ]:
def read_info(path = '../info/info.json'):
    global PATH, OUTPUT_PATH, PROCESSED_PATH, PERFIS_PATH, HORARIOS_PATH, PATH_MM, PATH_FAS, \
    DEG, MOR, PER, FAS, OUTPUT_HEADER,FAS_CONCAT, MM_CONCAT, PICKLES_CONCAT, OUT_INDEX
    with open(path) as f:
        info = json.load(f)
        PATH = info['paths']['fasoriais']['input']
        OUTPUT_PATH = info['paths']['fasoriais']['output']
        PROCESSED_PATH = info['paths']['fasoriais']['processed']
        PERFIS_PATH = info['paths']['perfis']['perfis']
        HORARIOS_PATH = info['paths']['perfis']['horarios']
        PATH_MM = info['paths']['mm']['input']
        PATH_FAS = info['paths']['fasoriais']['input']
        DEG = info['paths']['nn_models']['degrau']
        MOR = info['paths']['nn_models']['morro']
        PER = info['paths']['nn_models']['perfil']
        FAS = info['paths']['nn_models']['fasorial']
        FAS_CONCAT =  info['paths']['concat']['fas']
        MM_CONCAT =  info['paths']['concat']['mm'] 
        PICKLES_CONCAT=  info['paths']['concat']['pickles']        
        OUTPUT_HEADER = info['output']['header']
        OUT_INDEX = info['output']['header']

In [ ]:
read_info()

In [ ]:
OUT_CONCAT = '../outputs/concat/'
OUT = '../outputs/all/'

In [ ]:
#preguiça -- arrumar
out_index = OUT_INDEX

### Carregar modelos neurais

# Coleções principais

#### Lista principal

In [ ]:
global LISTA_CAD

#### Modelos neurais

In [ ]:
PER

In [ ]:
FAS = '../../fdetector/models/rn_hc_and/rn_hc_and'

In [ ]:
global PERFIL, DEGRAU, MORRO, NN_FAS
PERFIL, DEGRAU, MORRO, NN_FAS = load_models(PER, DEG, MOR, FAS)

#### Arquivos

In [ ]:
global files_fas, files_mm

#### Perfis e horários

In [ ]:
global HORARIOS, PERFIS, ATIVS, DPERFIS, DPERFIS_NORMAL, DPERFIS_TROCA

#### Estruturas auxiliares

In [ ]:
global dim, dmi, df_fas, dfiles_mm, dfiles_fas_troca, dfiles_mm_troca, dfiles_fas_normal, dfiles_mm_normal
global o_i_normal, o_i_troca, o_instas, list_i_normal, list_i_troca, list_intas

## Lista de medidores e lista por horários

In [ ]:
LISTA_CAD = get_list()

In [ ]:
HORARIOS, PERFIS, ATIVS = get_horario_info()

### Dicionários e listas

In [ ]:
dim, dmi = dicts(LISTA_CAD)

In [ ]:
#gls i_normal -> instalações normal
# i_troca -> instalações com troca
#o_i_normal -> ordenado instalações normal
o_i_normal = collections.OrderedDict(sorted(LISTA_CAD.get_users(index = 'instalacao', troca=0,tipo='dictionary').items()))
o_i_troca = collections.OrderedDict(sorted(LISTA_CAD.get_users(index = 'instalacao', troca=1,tipo='dictionary').items()))
o_instas =  collections.OrderedDict(sorted({**o_i_normal, **o_i_troca}.items()))

##### Dicionário com os horário de funcionamento de cada cliente

In [ ]:
DPERFIS = sdict()
for i in PERFIS.index:
    j = PERFIS.loc[i,'Ramo de Atividade']
    if HORARIOS.index.contains(j):
        DPERFIS[i] = list(HORARIOS.loc[j,:])
    else:
        DPERFIS[i] = list(HORARIOS.iloc[0,:])

DPERFIS_NORMAL={}
for i in DPERFIS:
    if i in o_i_normal:
        DPERFIS_NORMAL[i] = o_i_normal[i]

DPERFIS_TROCA={}
for i in DPERFIS:
    if i in o_i_troca:
        DPERFIS_TROCA[i] = o_i_troca[i]

In [ ]:
dfiles_fas, nf_fas=get_files_3(list(o_i_normal.keys()), dim,  PATH_FAS)
dfiles_mm, nf_mm = get_files_3(list(o_i_normal.keys()),dim , PATH_MM)

In [ ]:
dfiles_fas2 = deslistar(dfiles_fas)
dfiles_mm2 = deslistar(dfiles_mm)

In [ ]:
dfiles_fas_concat, nf_fas_concat = get_files_2( list(o_i_troca.keys()), FAS_CONCAT)
dfiles_mm_concat, nf_mm_concat = get_files_2( list(o_i_troca.keys()), MM_CONCAT)

## l_______________________________________________________________________

In [ ]:
#Criar dicionario de categorias
HORARIOS_PATH = '/home/ubuntu/data/data/maio_2019/horarios_v3.csv'
horarios = pd.read_csv(HORARIOS_PATH, sep=',')
dfcat = horarios.iloc[:,:2]
dfcat.set_index('nome',inplace=True)
scat = dfcat['cat']
dcat = scat.to_dict()

In [ ]:
ATIV = '/home/ubuntu/data/data/maio_2019/atividade_atividade_v2.csv'
dfativ = pd.read_csv(ATIV, sep=';', names = ['ramo', 'perfil'])
dfativ.set_index('ramo', inplace=True)
a2p = dfativ.loc[:,'perfil'].T.to_dict()
p2a = dict((v,k) for k,v in a2p.items())

In [ ]:
PERFIS[~PERFIS.loc[:,'cluster'].isin(horarios.loc[:,'nome'])]

## l_______________________________________________________________________

In [ ]:
dfiles_fas_concat = get_files_troca(dfiles_fas_concat)

In [ ]:
dfiles_mm_concat= get_files_troca(dfiles_mm_concat)

In [ ]:
odf_concat = collections.OrderedDict(sorted(dfiles_fas_concat.items()))

In [ ]:
odmm_concat = collections.OrderedDict(sorted(dfiles_mm_concat.items()))

In [ ]:
# Criar um dicionário único para todos os caminhos de arquivos
ddirs={}
ddirsn={}
ddirst={}
for u,v in dfiles_fas2.items():
    ddirs[u] = os.path.join(PATH_FAS,v)
    ddirsn[u] = os.path.join(PATH_FAS,v)
for u,v in dfiles_fas_concat.items():
    if u in ddirs:
        print('Warning. Instalacao repetida')
    ddirs[u] = os.path.join(FAS_CONCAT, v)
    ddirst[u] = os.path.join(FAS_CONCAT, v)

## l_______________________________________________________________________

In [ ]:
gt = pd.read_csv('../data/groundtruth.csv',sep=';',index_col=0)

## l_______________________________________________________________________

In [ ]:
mkdir -p /home/ubuntu/data/data/maio_2019_proc/concat/fas/

In [ ]:
mkdir -p /home/ubuntu/data/data/maio_2019_proc/fas-sep2/

In [ ]:
FAS_CONCAT_PROC  = '/home/ubuntu/data/data/maio_2019_proc/concat/fas/'

In [ ]:
PATH_FAS_PROC  = '/home/ubuntu/data/data/maio_2019_proc/fas-sep2/'

## l_______________________________________________________________________

In [ ]:
def procsave(d,path):
    shapes = {}
    f = {}
    print(len(d.items()))
    for n,(k,v) in enumerate(d.items()):
        print(n,end=' ')
        f[k]=fasorial(name=k)
        f[k].read_fasorial(v)
        raw_shape = f[k].df.shape
        f[k].proc_fasorial()
        shapes[i] = (raw_shape, f[k].df.shape)
        print(k)
        f[k].df.to_csv(os.path.join(path,str(k)+'.csv'))
    return f, shapes

In [ ]:
ft, st = procsave(ddirst,FAS_CONCAT_PROC)

In [ ]:
fn, sn = procsave(ddirsn,PATH_FAS_PROC)